In [1]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from konlpy.tag import Mecab
from wordcloud import WordCloud
from collections import Counter
from transformers import AutoTokenizer
from sentencepiece import SentencePieceTrainer, SentencePieceProcessor

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "../dataset"

train_df = pd.read_csv(f"{data_path}/cleaned_train.csv")
valid_df = pd.read_csv(f"{data_path}/cleaned_dev.csv")

In [3]:
print("Train DataFrame")
print(train_df.shape)
print(train_df.columns)

print("\nValid DataFrame")
print(valid_df.shape)
print(valid_df.columns)

Train DataFrame
(12457, 4)
Index(['fname', 'dialogue', 'summary', 'topic'], dtype='object')

Valid DataFrame
(499, 4)
Index(['fname', 'dialogue', 'summary', 'topic'], dtype='object')


# Sample Row

In [4]:
idx = random.randint(0, train_df.shape[0]-1)
sample = train_df.iloc[idx]
print(sample, "\n")

print(f"전체 대화 :\n{sample['dialogue']} \n")
print(f"요약 내용 :\n{sample['summary']}, \n")
print(f"대화 주제 :\n {sample['topic']}, \n")

fname                                              train_5426
dialogue    #Person1# 택시를 예약해 주실 수 있나요? #Person2# 개인 차량이 더...
summary     #Person1#은 개인 차량 대신 택시를 예약하도록 #Person2#에게 요청하여...
topic                                                   차량 예약
Name: 5426, dtype: object 

전체 대화 :
#Person1# 택시를 예약해 주실 수 있나요? #Person2# 개인 차량이 더 마음에 드실까요? #Person1# 개인 차량이요? 아니요, 괜찮습니다. #Person2# 개인 차량보다는 리무진이 더 좋습니다. 어떠세요? #Person1# 택시를 부탁드립니다. #Person2# 택시로 하겠습니다. 어디로 가시나요? #Person1# 록펠러 센터입니다. 택시를 바로 여기로 오게 할 수 있나요? #Person2# 택시는 곧 도착할 것입니다, 선생님. #Person1# 좋습니다. 제코트를 입고 내려가겠습니다. #Person2# 선생님이 준비되면 택시가 준비되어 있을 것입니다. 

요약 내용 :
#Person1#은 개인 차량 대신 택시를 예약하도록 #Person2#에게 요청하여 록펠러 센터로 향한다., 

대화 주제 :
 차량 예약, 



In [5]:
sample = train_df.sample(10)
sample.to_csv("../dataset/cleaned_sample.csv", index=False)

In [6]:
train_df['is_train'] = 1
valid_df['is_train'] = 0
total_df = pd.concat([train_df, valid_df], ignore_index=True)

In [7]:
def get_frequent_words(text, n=1000):
    mecab = Mecab()
    words = []
    for sentence in text:
        words.extend(mecab.nouns(sentence))
    return [word for word, _ in Counter(words).most_common(n)]

frequent_words = get_frequent_words(total_df['dialogue'])

In [8]:
mecab = Mecab()
def mecab_tokenize(text):
    return ' '.join(mecab.morphs(text))

def train_sentencepiece_with_mecab(text, vocab_size=1077, model_prefix='sp_model'):
    preprocessed_text = [mecab_tokenize(sentence) for sentence in text]
    
    with open('temp_mecab.txt', 'w', encoding='utf-8') as f:
        for line in preprocessed_text:
            f.write(line + '\n')
    
    SentencePieceTrainer.Train(
        f'--input=temp_mecab.txt --model_prefix={model_prefix} '
        f'--vocab_size={vocab_size} --character_coverage=0.9995 '
        f'--split_by_whitespace=true')
    
    sp = SentencePieceProcessor()
    sp.Load(f'{model_prefix}.model')
    
    return [sp.IdToPiece(id) for id in range(sp.GetPieceSize())]

sentencepiece_tokens = train_sentencepiece_with_mecab(total_df['dialogue'])
print(f"SentencePiece tokens: {sentencepiece_tokens[:100]}")
print(f"Total SentencePiece tokens: {len(sentencepiece_tokens)}")

SentencePiece tokens: ['<unk>', '<s>', '</s>', '▁', 's', '요', '있', '를', '것', '이', '리', '게', '지', '신', '스', '기', '나', '않', '가', '서', '어', '다', '일', '상', '러', '시', '라', '드', '로', '자', '금', '도', '야', '인', '장', '구', '사', '분', '싶', '까', '니', '면', '생', '안', '우', '람', '문', '고', '운', '원', '히', '소', '데', '터', '두', '미', '직', '트', '려', '든', '하', '학', '전', '음', '진', '디', '아', '님', '대', '무', '해', '실', '용', '부', '녀', '공', '래', '치', '으', '연', '크', '교', '5', '명', '식', '유', '후', '제', '세', '성', '영', '위', '주', '행', '관', '엇', '화', '들', '레', '경']
Total SentencePiece tokens: 1077


sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=temp_mecab.txt --model_prefix=sp_model --vocab_size=1077 --character_coverage=0.9995 --split_by_whitespace=true
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: temp_mecab.txt
  input_format: 
  model_prefix: sp_model
  model_type: UNIGRAM
  vocab_size: 1077
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  

In [11]:
def refine_tokens(frequent_words, sentencepiece_tokens):
    refined_tokens = set()
    for token in sentencepiece_tokens:
        token_without_underscore = token.replace('▁', '')
        if token_without_underscore in frequent_words or token in frequent_words:
            refined_tokens.add(token)
    
    # EDA 결과(frequent words)에서 SentencePiece 토큰과 겹치지 않는 단어들 추가
    for word in frequent_words:
        if word not in refined_tokens and f'▁{word}' not in refined_tokens:
            refined_tokens.add(word)
    
    return list(refined_tokens)

refined_tokens = refine_tokens(frequent_words, sentencepiece_tokens)
print(refined_tokens)

['날', '위험', '컵', '화요일', '피아노', '청구', '주인', '사이', '길', '걱정', '화장실', '운동', '불', '대신', '파운드', '일부', '감기', '텐데', '결혼', '페이지', '경찰', '끝', '왕', '고양이', '자동차', '병', '그곳', '경기', '구입', '코트', '프로젝트', '상자', '기업', '시스템', '숙제', '마일', '금액', '복용', '빵', '활동', '입학', '교통', '규칙', '언어', '최소', '성과', '티켓', '블록', '색상', '베이징', '여행', '비행', '보통', '회의', '이쪽', '에너지', '내용', '사이트', '포기', '뭔가', '처음', '언니', '전문', '구매', '휴가', '숨', '기사', '운', '상사', '매니저', '주차', '시기', '정류장', '결정', '타이핑', '연락', '유용', '보고서', '치즈', '클래식', '예술', '세트', '이상', '외국', '층', '모습', '대학교', '내일', '퍼센트', '꽃', '안전', '삶', '과목', '뭘', '명함', '생선', '이해', '의사', '목', '추가', '그걸로', '동안', '제출', '인생', '미안', '흡연', '아들', '애', '종류', '백', '근무', '쪽', '통과', '코', '이틀', '가이드', '며칠', '면접', '나이', '가격', '축구', '산책', '화', '기차', '방문', '유지', '초대', '통', '주문', '아내', '기타', '색', '일요일', '아래', '졸업', '나무', '편리', '교육', '더블', '능력', '문서', '최근', '아이', '가게', '제니', '채소', '해결', '삼촌', '시험', '사무실', '다이어트', '신발', '인터넷', '등록', '연주', '신문', '현재', '명', '일본', '마찬가지', '수표', '개', '옆', '가입', '기록', '대만',

In [13]:
def remove_similar_tokens(tokens, similarity_threshold=0.8):
    def similarity(a, b):
        return len(set(a) & set(b)) / float(len(set(a) | set(b)))
    
    unique_tokens = []
    for token in tokens:
        if not any(similarity(token, t) > similarity_threshold for t in unique_tokens):
            unique_tokens.append(token)
    
    return unique_tokens

final_tokens = remove_similar_tokens(refined_tokens)
print(final_tokens)

['날', '위험', '컵', '화요일', '피아노', '청구', '주인', '사이', '길', '걱정', '화장실', '운동', '불', '대신', '파운드', '일부', '감기', '텐데', '결혼', '페이지', '경찰', '끝', '왕', '고양이', '자동차', '병', '그곳', '경기', '구입', '코트', '프로젝트', '상자', '기업', '시스템', '숙제', '마일', '금액', '복용', '빵', '활동', '입학', '교통', '규칙', '언어', '최소', '성과', '티켓', '블록', '색상', '베이징', '여행', '비행', '보통', '회의', '이쪽', '에너지', '내용', '사이트', '포기', '뭔가', '처음', '언니', '전문', '구매', '휴가', '숨', '기사', '운', '상사', '매니저', '주차', '시기', '정류장', '결정', '타이핑', '연락', '유용', '보고서', '치즈', '클래식', '예술', '세트', '이상', '외국', '층', '모습', '대학교', '내일', '퍼센트', '꽃', '안전', '삶', '과목', '뭘', '명함', '생선', '이해', '의사', '목', '추가', '그걸로', '동안', '제출', '인생', '미안', '흡연', '아들', '애', '종류', '백', '근무', '쪽', '통과', '코', '이틀', '가이드', '며칠', '면접', '나이', '가격', '축구', '산책', '화', '기차', '방문', '유지', '초대', '통', '주문', '아내', '기타', '색', '일요일', '아래', '졸업', '나무', '편리', '교육', '더블', '능력', '문서', '최근', '아이', '가게', '제니', '채소', '해결', '삼촌', '시험', '사무실', '다이어트', '신발', '인터넷', '등록', '연주', '신문', '현재', '명', '일본', '마찬가지', '수표', '개', '옆', '가입', '기록', '대만',

In [17]:
model_name = "EbanLee/kobart-summary-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_tokens(final_tokens)

print(f"Updated vocabulary size: {len(tokenizer)}")
print(tokenizer.special_tokens_map)

Updated vocabulary size: 30507
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


In [15]:
tokenizer.save_pretrained("../tokenizer")

('../tokenizer/tokenizer_config.json',
 '../tokenizer/special_tokens_map.json',
 '../tokenizer/tokenizer.json')